In [ ]:
import uproot
import polars as pl
print("uproot version: ", uproot.__version__)

import tempfile
import subprocess

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.file_locations import data_files_location, intermediate_files_location

import matplotlib.pyplot as plt
import numpy as np

import ast


In [ ]:
# load presel_weights_df.parquet
#presel_weights_df = pl.read_parquet(f"{intermediate_files_location}/presel_weights_df.parquet")
#presel_weights_df


In [ ]:
num_events = 1000


In [ ]:
# uproot works for the CV weights, but not the systematic weights
f = uproot.open(data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root")


In [ ]:
f["nuselection"]["NeutrinoSelectionFilter"].items()

In [ ]:
weight_spline_times_tune = f["nuselection"]["NeutrinoSelectionFilter"]["weightSplineTimesTune"].array(library="np", entry_stop=num_events)
weight_cv = f["wcpselection"]["T_eval"]["weight_cv"].array(library="np", entry_stop=num_events)
weight_spline = f["wcpselection"]["T_eval"]["weight_spline"].array(library="np", entry_stop=num_events)
weight_cv_times_spline = weight_cv * weight_spline


bins = np.linspace(0, 5, 100)

plt.figure()
plt.hist(weight_spline_times_tune, bins=bins, histtype="step", label="Pandora weightSplineTimesTune")
plt.hist(weight_cv_times_spline, bins=bins, histtype="step", label="Wire-Cell weight_cv * weight_spline")
plt.legend()
plt.xlabel("Weight")
plt.yscale("log")
plt.show()

diffs = weight_spline_times_tune - weight_cv_times_spline
diffs = np.clip(diffs, -10, 10)

plt.figure()
plt.hist(diffs, bins=100, histtype="step")
plt.xlabel("Pandora weightSplineTimesTune - Wire-Cell weight_cv * weight_spline")
plt.yscale("log")
plt.show()


In [ ]:
# vector variables not available from uproot sadly:
"""
f = uproot.open(data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root")["nuselection"]["NeutrinoSelectionFilter"]

print("\nweights.first",f["weights.first"].members)
print("\nweights.second",f["weights.second"].members)
print("\nweightsFlux",f["weightsFlux"].members)
print("\nweightsGenie",f["weightsGenie"].members)
print("\nweightsReint",f["weightsReint"].members)
print("\nweightSpline",f["weightSpline"].members)
print("\nweightTune",f["weightTune"].members)
print("\nweightSplineTimesTune",f["weightSplineTimesTune"].members)
"""
pass

In [ ]:
def _cxx_escape(s: str) -> str:
    return s.replace('\\', '\\\\').replace('"', '\\"')

def get_weights(
    file_path: str,
    tree_path: str = "nuselection/NeutrinoSelectionFilter",
    branch_name: str = "weightsGenie",
    max_entries: int = -1,
    root_bin: str = "root",
) -> list[list[int]]:
    cpp_macro = r'''
#include <TFile.h>
#include <TTree.h>
#include <TBranch.h>
#include <TROOT.h>
#include <TSystem.h>
#include <vector>
#include <iostream>

void extract_vector_ushort(const char* filePath, const char* treePath, const char* branchName, Long64_t maxEntries=-1) {
    TFile* f = TFile::Open(filePath);
    if (!f || f->IsZombie()) { std::cout << "__ERROR__ Cannot open file\n"; return; }
    TObject* obj = f->Get(treePath);
    TTree* t = dynamic_cast<TTree*>(obj);
    if (!t) { std::cout << "__ERROR__ Tree not found at path\n"; return; }
    TBranch* br = t->GetBranch(branchName);
    if (!br) { std::cout << "__ERROR__ Branch not found\n"; return; }

    std::vector<unsigned short>* vec = nullptr;
    t->SetBranchAddress(branchName, &vec);

    Long64_t nentries = t->GetEntries();
    if (maxEntries >= 0 && maxEntries < nentries) nentries = maxEntries;

    std::cout << "__BEGIN__\n";
    for (Long64_t i = 0; i < nentries; ++i) {
        t->GetEntry(i);
        if (!vec) { std::cout << "[]\n"; continue; }
        std::cout << "[";
        for (size_t j = 0; j < vec->size(); ++j) {
            if (j) std::cout << ",";
            std::cout << static_cast<unsigned int>(vec->at(j));
        }
        std::cout << "]\n";
    }
    std::cout << "__END__\n";
}
'''
    file_path_cxx = _cxx_escape(os.path.abspath(file_path))
    tree_path_cxx = _cxx_escape(tree_path)
    branch_name_cxx = _cxx_escape(branch_name)

    with tempfile.TemporaryDirectory() as td:
        macro_path = os.path.join(td, "extract_vector_ushort.C")
        with open(macro_path, "w") as f:
            f.write(cpp_macro)

        arg_expr = f'{macro_path}("{file_path_cxx}","{tree_path_cxx}","{branch_name_cxx}",{int(max_entries)})'
        cmd = [root_bin, "-l", "-b", "-q", arg_expr]

        proc = subprocess.run(cmd, text=True, capture_output=True)
        combined = (proc.stdout or "") + "\n" + (proc.stderr or "")

        begin = "__BEGIN__"
        end = "__END__"
        if begin not in combined or end not in combined:
            raise RuntimeError(f"Failed to parse ROOT output.\nReturn code: {proc.returncode}\n--- stdout ---\n{proc.stdout}\n--- stderr ---\n{proc.stderr}")

        payload = combined.split(begin, 1)[1].split(end, 1)[0]
        rows: list[list[int]] = []
        for line in payload.strip().splitlines():
            line = line.strip()
            if not line:
                continue
            try:
                arr = ast.literal_eval(line)
                if isinstance(arr, list):
                    rows.append(np.array([int(x) / 1000. for x in arr]))
            except Exception:
                pass
        return np.array(rows)



In [ ]:

# factor of 1000 for integer -> float conversion exists here: https://github.com/ubneutrinos/searchingfornues/blob/c1d8558e1990d9553b874daf9807c15e6ad8dc5e/Selection/AnalysisTools/EventWeightTree_tool.cc#L245
weights_genie = get_weights(
    data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root",
    branch_name="weightsGenie",
    max_entries=num_events,
)
weights_flux = get_weights(
    data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root",
    branch_name="weightsFlux",
    max_entries=num_events,
)
weights_reint = get_weights(
    data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root",
    branch_name="weightsReint",
    max_entries=num_events,
)

wc_kine_reco_Enus = f["wcpselection"]["T_KINEvars"]["kine_reco_Enu"].array(library="np", entry_stop=num_events)


In [ ]:
import ROOT

def get_weights_pairs(file_path, tree_path="nuselection/NeutrinoSelectionFilter", branch_name="weights", max_entries=-1):

    print("opening file..")
    f = ROOT.TFile.Open(file_path)
    if not f or f.IsZombie():
        raise RuntimeError("Cannot open file")

    print("getting tree..")
    
    t = f.Get(tree_path)
    if not t:
        raise RuntimeError("Tree not found")

    print("t: ", t)
    
    nentries = t.GetEntries()
    if max_entries >= 0 and max_entries < nentries:
        nentries = max_entries

    print("nentries: ", nentries)
    
    rows = []
    for i in range(nentries):
        print("current entry: ", i)
        t.GetEntry(i)
        weights_map = getattr(t, branch_name)
        print("weights_map: ", weights_map)
        
        entry_data = []
        for key, values in weights_map:
            for val in values:
                entry_data.append([key, val])
                print("adding: ", key, val)
        rows.append(entry_data)
    
    f.Close()
    return rows

In [ ]:
weights_other = get_weights_pairs(
    data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root",
    max_entries=10,
)

weights_other

In [ ]:
print(1/0)

In [ ]:
bins = np.linspace(0, 2000, 11)

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(10):
    curr_event_weights = weights_genie[:, uni_i]
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"GENIE_All uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(10):
    curr_event_weights = weights_flux[:, uni_i] * weight_spline_times_tune
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"Flux uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(10):
    curr_event_weights = weights_reint[:, uni_i] * weight_spline_times_tune
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"Reint uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()
